In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd gdrive/My\ Drive/EVA

/content/gdrive/My Drive/EVA


In [0]:
cd session19/

/content/gdrive/My Drive/EVA/session19


In [0]:
#cd session19/

In [0]:
from __future__ import print_function, division
import scipy

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate
from keras.layers import BatchNormalization, Activation, ZeroPadding2D, Add
from keras.layers.advanced_activations import PReLU, LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.applications import VGG19
from keras.models import Sequential, Model
from keras.optimizers import Adam
import datetime
import matplotlib.pyplot as plt
import sys
#from data_loader import DataLoader
import numpy as np
import os

Using TensorFlow backend.


In [0]:
pip install scipy==1.1.0

In [0]:
import scipy
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

In [0]:
class DataLoader():
    def __init__(self,img_res=(64,64)):
        self.img_res = img_res

    def load_data(self, batch_size=1, is_testing=False):      
        path = glob('images/')
        x_datasets = []
        for sample11 in os.listdir('images'):
          x_datasets.append(sample11)

        batch_images = np.random.choice(x_datasets, size=batch_size)

        imgs_hr = []
        for img_path in batch_images:
            img = self.imread('datasets/'+img_path)
            img_hr = scipy.misc.imresize(img, self.img_res)
            imgs_hr.append(img_hr)

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        print(imgs_hr)
        return imgs_hr

    def imread(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)

In [0]:
class GAN():
    def __init__(self):
        self.img_rows = 64
        self.img_cols = 64
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss='binary_crossentropy',
            optimizer=optimizer,
            metrics=['accuracy'])

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        z = Input(shape=(self.latent_dim,))
        img = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated images as input and determines validity
        validity = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.combined = Model(z, validity)
        self.combined.compile(loss='binary_crossentropy', optimizer=optimizer)


    def build_generator(self):

        model = Sequential()

        model.add(Dense(256, input_dim=self.latent_dim))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(1024))
        model.add(LeakyReLU(alpha=0.2))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Dense(np.prod(self.img_shape), activation='tanh'))
        model.add(Reshape(self.img_shape))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Flatten(input_shape=self.img_shape))
        model.add(Dense(512))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(256))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dense(1, activation='sigmoid'))
        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def load_data_new(self, batch_size=1,img_res=(128, 128)):      
        path = glob('images/')
        x_datasets = []
        for sample11 in os.listdir('images'):
          x_datasets.append(sample11)

        batch_images = np.random.choice(x_datasets, size=batch_size)

        imgs_hr = []
        for img_path in batch_images:
            img = self.imread_new('images/'+img_path)
            img_hr = scipy.misc.imresize(img,img_res)
            imgs_hr.append(img_hr)

        imgs_hr = np.array(imgs_hr) / 127.5 - 1.
        return imgs_hr

    def imread_new(self, path):
        return scipy.misc.imread(path, mode='RGB').astype(np.float)


    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        #(X_train, _), (_, _) = mnist.load_data()


        X_train = self.load_data_new(batch_size,img_res=(64,64))
        print(X_train.shape)
        # Rescale -1 to 1
        #X_train = X_train / 127.5 - 1.
        #X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        for epoch in range(epochs):

            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of images
            idx = np.random.randint(0, X_train.shape[0], batch_size)
            imgs = X_train[idx]

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_imgs = self.generator.predict(noise)

            # Train the discriminator
            d_loss_real = self.discriminator.train_on_batch(imgs, valid)
            d_loss_fake = self.discriminator.train_on_batch(gen_imgs, fake)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            g_loss = self.combined.train_on_batch(noise, valid)

            # Plot the progress
            print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

            # If at save interval => save generated image samples
            if epoch % sample_interval == 0:
                self.sample_images(epoch)

    def sample_images(self, epoch):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0])
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig("code0_guns_images/mnist_%d.png" % epoch)
        plt.close()

In [0]:
if __name__ == '__main__':
    gan = GAN()
    gan.train(epochs=30000, batch_size=32, sample_interval=200)




Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 12288)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               6291968   
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 6,423,553
Trainable params: 6,423,553
Non-trainable params: 0
_________________________________________

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


(32, 64, 64, 3)










/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.834842, acc.: 31.25%] [G loss: 0.714589]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Streaming output truncated to the last 5000 lines.
25001 [D loss: 0.115607, acc.: 95.31%] [G loss: 3.320995]
25002 [D loss: 0.144329, acc.: 95.31%] [G loss: 3.657969]
25003 [D loss: 0.150470, acc.: 95.31%] [G loss: 3.435773]
25004 [D loss: 0.061211, acc.: 98.44%] [G loss: 4.050161]
25005 [D loss: 0.068651, acc.: 98.44%] [G loss: 4.168788]
25006 [D loss: 0.159345, acc.: 96.88%] [G loss: 3.630758]
25007 [D loss: 0.289755, acc.: 92.19%] [G loss: 3.028955]
25008 [D loss: 0.209256, acc.: 92.19%] [G loss: 3.250784]
25009 [D loss: 0.243544, acc.: 90.62%] [G loss: 3.637341]
25010 [D loss: 0.221739, acc.: 92.19%] [G loss: 3.509367]
25011 [D loss: 0.268004, acc.: 89.06%] [G loss: 3.272502]
25012 [D loss: 0.276974, acc.: 90.62%] [G loss: 2.788328]
25013 [D loss: 0.122719, acc.: 96.88%] [G loss: 3.093438]
25014 [D loss: 0.185252, acc.: 93.75%] [G loss: 3.436696]
25015 [D loss: 0.157595, acc.: 96.88%] [G loss: 3.262198]
25016 [D loss: 0.222720, acc.: 92.19%] [G loss: 3.245239]
25017 [D loss: 0.1396

In [0]:
!ls

2020-03-04_04:26:27  code0_guns_images	    grey_images        mnist_200.png
2020-03-04_04:26:56  code0_guns_images_2    images	       Untitled0.ipynb
2020-03-04_04:48:48  code0_guns_images_ols  images2	       Untitled1.ipynb
93_g.jpg	     code0_guns.ipynb	    images3	       Untitled2.ipynb
93.jpg		     _data		    load_images.ipynb  Untitled3.ipynb
93_r.jpg	     dcgan		    mnist_0.png


In [0]:
dd = 0

for xyx in os.listdir('images'):
  dd = dd + 1

dd

185

In [0]:
gan.train(epochs=30000, batch_size=132, sample_interval=200)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:94: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:87: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


(132, 64, 64, 3)
0 [D loss: 0.102468, acc.: 98.11%] [G loss: 3.832927]


/usr/local/lib/python3.6/dist-packages/keras/engine/training.py:493: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.215369, acc.: 95.83%] [G loss: 3.573179]
2 [D loss: 0.226964, acc.: 95.83%] [G loss: 3.449694]
3 [D loss: 0.085700, acc.: 98.11%] [G loss: 3.633639]
4 [D loss: 0.083443, acc.: 97.73%] [G loss: 3.413011]
5 [D loss: 0.045743, acc.: 99.24%] [G loss: 3.988087]
6 [D loss: 0.081566, acc.: 98.11%] [G loss: 3.976997]
7 [D loss: 0.089378, acc.: 97.73%] [G loss: 3.687162]
8 [D loss: 0.106662, acc.: 97.35%] [G loss: 3.642652]
9 [D loss: 0.090419, acc.: 97.73%] [G loss: 3.624517]
10 [D loss: 0.074465, acc.: 98.11%] [G loss: 3.688493]
11 [D loss: 0.071869, acc.: 98.11%] [G loss: 3.986613]
12 [D loss: 0.114521, acc.: 96.59%] [G loss: 3.717269]
13 [D loss: 0.136378, acc.: 95.83%] [G loss: 3.379537]
14 [D loss: 0.066123, acc.: 98.11%] [G loss: 3.832035]
15 [D loss: 0.069436, acc.: 97.35%] [G loss: 4.121146]
16 [D loss: 0.119665, acc.: 96.59%] [G loss: 3.543912]
17 [D loss: 0.055167, acc.: 98.86%] [G loss: 3.888249]
18 [D loss: 0.067659, acc.: 98.11%] [G loss: 3.903110]
19 [D loss: 0.04733